In [1]:
import os
from ultralytics import NAS
os.chdir("..")
ROOT = os.getcwd()
print(ROOT)


Matplotlib created a temporary cache directory at /localscratch/2183901/matplotlib-oqrrumcj because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


/home/mautushid/Cowsformer


In [2]:
import os
os.environ['TORCH_HOME'] = '/home/mautushid/.torch'

In [3]:
from models.nas import *


The console stream is logged into /home/mautushid/sg_logs/console.log


[2024-03-01 15:45:20] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-03-01 15:45:21] WARNING - __init__.py - Failed to import pytorch_quantization
[2024-03-01 15:45:30] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-03-01 15:45:30] WARNING - export.py - Failed to import pytorch_quantization
[2024-03-01 15:45:30] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


In [4]:
### class inits and other inputs

path_model = 'yolo_nas_l' 
dir_train = "/home/mautushid/Cowsformer/data/cow200/yolov5/train"
dir_val = "/home/mautushid/Cowsformer/data/cow200/yolov5/val"
dir_test = "/home/mautushid/Cowsformer/data/cow200/yolov5/test/test"
name_task = "cow200"
data_yaml_path = "/home/mautushid/Cowsformer/data/cow200/yolov5/data.yaml"

#data_yaml_path = "/home/mautushid/Cowsformer/data/cow200/yolov5/data.yaml"
finetuned_model_path = "/home/mautushid/Cowsformer/checkpoints_cow200/n200_yolo_nas_l_i12/RUN_20240224_060546_568311/ckpt_best.pth"

### Creating instance of Niche_YOLO_NAS class
my_nas = Niche_YOLO_NAS(path_model, dir_train, dir_val, dir_test, name_task)


[2024-03-01 15:45:32] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-03-01 15:45:32] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


In [5]:
### load finetuned model
best_model = my_nas.load(finetuned_model_path)

model /home/mautushid/Cowsformer/checkpoints_cow200/n200_yolo_nas_l_i12/RUN_20240224_060546_568311/ckpt_best.pth loaded


In [6]:
my_nas.evaluate_trained_model(best_model, data_yaml_path, "test")

[2024-03-01 15:45:35] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Testing:  33%|███▎      | 1/3 [00:02<00:05,  2.62s/it]

{'Precision@0.50': 0.0,
 'Recall@0.50': 0.0,
 'mAP@0.50': 0.0,
 'F1@0.50': 0.0,
 'Best_score_threshold': 0.0,
 'Precision@0.50:0.95': 0.0,
 'Recall@0.50:0.95': 0.0,
 'mAP@0.50:0.95': 0.0,
 'F1@0.50:0.95': 0.0}

Testing: 100%|██████████| 3/3 [00:02<00:00,  1.04it/s]


In [7]:
#my_nas.get_evaluation_matrix(best_model, data_yaml_path)